In [1]:
from tqdm import tqdm
import os
import time
from datetime import datetime
from random import randint

import numpy as np
from scipy import stats
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import KFold

import nibabel as nib

import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.animation as anim
import matplotlib.patches as mpatches
import matplotlib.gridspec as gridspec

import seaborn as sns
from skimage.transform import resize
from skimage.util import montage

from IPython.display import Image as show_gif
from IPython.display import clear_output

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from torch.optim import Adam, AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau

import re
import warnings


warnings.simplefilter("ignore")


In [2]:
from utils.Meter import dice_coef_metric_per_classes, jaccard_coef_metric_per_classes

# CAN POTENTIALLY TRY DIFFERENT AUGMENTATION TECHNIQUES HERE
from utils.BratsDataset import BratsDataset

from utils.Meter import BCEDiceLoss

from models.UNet3d import UNet3d
from models.UNet3dSingleConv import UNet3dSingleConv
from models.UNet3dDropout import UNet3dDropout
from models.SwinUNETR import SwinUNETR
from models.UNet3d_your_modifications import UNet3d_your_modifications
from models.UNet3d_SELU import UNet3d_SELU
from models.UNet3d_atten import UNet3d_atten
from models.ONet3d import ONet3d
from models.ONet3d_v2 import ONet3d_v2
from models.ONet3d_v3 import ONet3d_v3
from models.ONet3d_v3_DoubleConv import ONet3d_v3_DoubleConv
from models.UNet3d_GELU import UNet3d_GELU
from models.ONet3d_v3_GELU import ONet3d_v3_GELU
from models.SphereNet3d import SphereNet3d
from models.SphereNet3d import SphereNet3d
# from models.SphereNet3d_GELU import SphereNet3d_GELU


In [3]:
def get_dataloaders(
    dataset: torch.utils.data.Dataset,
    path_to_csv: str,
    # phase: str,
    val_fold: int = 0,  # Choose which fold to be the validation fold
    test_fold: int = 1,
    batch_size: int = 1,
    num_workers: int = 4,
    do_resizing: bool = True,
):
    assert (val_fold != test_fold)

    df = pd.read_csv(path_to_csv)

    '''Returns: dataloader for the model training'''
    # Data in folds other than 0 are used for training
    train_df = df.loc[~df['fold'].isin(
        [val_fold, test_fold])].reset_index(drop=True)
    # Data in fold 0 is used for validation
    val_df = df.loc[df['fold'] == val_fold].reset_index(drop=True)
    test_df = df.loc[df['fold'] == test_fold].reset_index(drop=True)

    # dataset = dataset(df, phase)
    train_dataset = dataset(train_df, "train", do_resizing=do_resizing)
    val_dataset = dataset(val_df, "val", do_resizing=do_resizing)
    test_dataset = dataset(test_df, "test", do_resizing=do_resizing)
    train_dataloader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=True,
    )
    val_dataloader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=True,
    )
    test_dataloader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        shuffle=True,
    )
    return train_dataloader, val_dataloader, test_dataloader


In [4]:
modelDict = {
    "3DOnet_DoubleConv_Kernel1": ONet3d_v3_DoubleConv(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DOnet_SingleConv_Kernel1": ONet3d_v3(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DOnet_SingleConv_Kernel1_32_Channels": ONet3d_v3(in_channels=4, n_classes=3, n_channels=32).to('cuda'),
    "3DOnet_SingleConv_Kernel1_GELU": ONet3d_v3_GELU(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DOnet_SingleConv_Kernel1_GELU_AdamW": ONet3d_v3_GELU(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DOnet_SingleConv_Kernel3": ONet3d_v2(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DOnet_SingleConv_Kernel5": ONet3d(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DUnet": UNet3d(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DUnet_32_Channels": UNet3d(in_channels=4, n_classes=3, n_channels=32).to('cuda'),
    "3DUnet_Atten": UNet3d_atten(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DUnet_Dropout": UNet3dDropout(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DUnet_GELU": UNet3d_GELU(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DUnet_SELU": UNet3d_SELU(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "3DUnet_SingleConv": UNet3dSingleConv(in_channels=4, n_classes=3, n_channels=24).to('cuda'),
    "SphereNet3D": SphereNet3d(in_channels=4, n_classes=3, n_channels=16).to('cuda'),
    "SwinUNETR": SwinUNETR(in_channels=4, out_channels=3, img_size=(128, 224, 224), depths=(1, 1, 1, 1), num_heads=(2, 4, 8, 16)).to('cuda'),
    "SwinUNETR_AdamW": SwinUNETR(in_channels=4, out_channels=3, img_size=(128, 224, 224), depths=(1, 1, 1, 1), num_heads=(2, 4, 8, 16)).to('cuda'),
    "SwinUNETR_DoubleLayerDepth": SwinUNETR(in_channels=4, out_channels=3, img_size=(128, 224, 224), depths=(2, 2, 2, 2), num_heads=(2, 4, 8, 16)).to('cuda'),
  }

In [5]:
def compute_scores_per_classes(model,
                               dataloader,
                               classes):
    """
    Compute Dice and Jaccard coefficients for each class.
    Params:
        model: neural net for make predictions.
        dataloader: dataset object to load data from.
        classes: list with classes.
        Returns: dictionaries with dice and jaccard coefficients for each class for each slice.
    """
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    dice_scores_per_classes = {key: list() for key in classes}
    iou_scores_per_classes = {key: list() for key in classes}
    ids = {"Ids":list()}

    with torch.no_grad():
        for i, data in enumerate(dataloader):
            imgs, targets = data['image'], data['mask']
            imgs, targets = imgs.to(device), targets.to(device)
            logits = model(imgs)
            logits = logits.detach().cpu().numpy()
            targets = targets.detach().cpu().numpy()

            dice_scores = dice_coef_metric_per_classes(logits, targets)
            iou_scores = jaccard_coef_metric_per_classes(logits, targets)
            ids["Ids"].extend(data["Id"])

            for key in dice_scores.keys():
                dice_scores_per_classes[key].extend(dice_scores[key])

            for key in iou_scores.keys():
                iou_scores_per_classes[key].extend(iou_scores[key])
    return dice_scores_per_classes, iou_scores_per_classes, ids


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [6]:

  
def evaluate_all(chkpt_type="best", results_dict = {}):
  chkpt_prefix = "your_best_model" if chkpt_type == "best" else "your_last_epoch_model"
  for dirname in os.listdir("Logs"):
    start = datetime.now()
    model_name = dirname
    model = modelDict[model_name]
    results_dict[model_name] = {}
    
    checkpoint_path = None
    
    for filename in os.listdir(f"Logs/{dirname}"):
      if chkpt_prefix in filename:
        checkpoint_path = f"Logs/{dirname}/{filename}"
        break
    
    try:
      model.load_state_dict(torch.load(
          checkpoint_path, map_location='cpu'))
      model.eval()
      print(f"{model_name} loaded with chkpt: {checkpoint_path}. parameters: {count_parameters(model)}")
    except Exception as e:
      print(f"Error loading {model_name} with chkpt: {checkpoint_path}. parameters: {count_parameters(model)}")
      print(e)
      
    _, _, test_dataloader = get_dataloaders(
        dataset=BratsDataset, path_to_csv="./fold_data.csv", val_fold=0, test_fold=1, batch_size=1, do_resizing=True)
    
    dice_scores_per_classes, iou_scores_per_classes, ids = compute_scores_per_classes(
        model, test_dataloader, ['WT', 'TC', 'ET']
    )
    ids_df = pd.DataFrame(ids)
    ids_df.columns = ['Ids']
    
    dice_df = pd.DataFrame(dice_scores_per_classes)


    dice_df.columns = ['WT dice', 'TC dice', 'ET dice']

    iou_df = pd.DataFrame(iou_scores_per_classes)
    iou_df.columns = ['WT jaccard', 'TC jaccard', 'ET jaccard']
    val_metics_df = pd.concat([ids_df, dice_df, iou_df], axis=1, sort=True)
    val_metics_df = val_metics_df.loc[:, ['Ids',
                                          'WT dice', 'WT jaccard',
                                          'TC dice', 'TC jaccard',
                                          'ET dice', 'ET jaccard']]
    mean_metrics_df = val_metics_df.mean()
    val_metics_df.sort_values(by=["Ids"], inplace=True)
    val_metics_df = val_metics_df.reset_index(drop=True)
    results_dict[model_name]["val_metics_df"] = val_metics_df
    results_dict[model_name]["WT dice"] = mean_metrics_df["WT dice"]
    results_dict[model_name]["WT jaccard"] = mean_metrics_df["WT jaccard"]
    results_dict[model_name]["TC dice"] = mean_metrics_df["TC dice"]
    results_dict[model_name]["TC jaccard"] = mean_metrics_df["TC jaccard"]
    results_dict[model_name]["ET dice"] = mean_metrics_df["ET dice"]
    results_dict[model_name]["ET jaccard"] = mean_metrics_df["ET jaccard"]
    inference_time = datetime.now() - start
    results_dict[model_name]["Inference time"] = inference_time
    
    del model
  return results_dict





In [7]:
eval_dict_last_epoch = evaluate_all(chkpt_type="last")


3DOnet_SingleConv_Kernel1_32_Channels loaded with chkpt: Logs/3DOnet_SingleConv_Kernel1_32_Channels/your_last_epoch_model_20230314-222549.pth. parameters: 5597571
SphereNet3D loaded with chkpt: Logs/SphereNet3D/your_last_epoch_model_20230319-011358.pth. parameters: 6134019
3DUnet_SingleConv loaded with chkpt: Logs/3DUnet_SingleConv/your_last_epoch_model_20230322-164951.pth. parameters: 3006507
3DOnet_SingleConv_Kernel1_GELU_AdamW loaded with chkpt: Logs/3DOnet_SingleConv_Kernel1_GELU_AdamW/your_last_epoch_model_20230317-195004.pth. parameters: 3150435
SwinUNETR_AdamW loaded with chkpt: Logs/SwinUNETR_AdamW/your_last_epoch_model_20230311-072307.pth. parameters: 14981601
3DUnet_Atten loaded with chkpt: Logs/3DUnet_Atten/your_last_epoch_model_20230311-222850.pth. parameters: 6094231
3DOnet_SingleConv_Kernel1 loaded with chkpt: Logs/3DOnet_SingleConv_Kernel1/your_last_epoch_model_20230313-220534.pth. parameters: 3150435
SwinUNETR loaded with chkpt: Logs/SwinUNETR/your_last_epoch_model_2023

KeyboardInterrupt: 

In [ ]:
eval_dict_best_epoch = evaluate_all(chkpt_type="best")


In [ ]:
with open("results/results_last_epoch.txt", "w") as outfile:
    outfile.write(str(eval_dict_last_epoch))
    
with open("results/results_best_epoch.txt", "w") as outfile:
    outfile.write(str(eval_dict_best_epoch))


In [ ]:
eval_dict_last_epoch_trunc = eval_dict_last_epoch.copy()
for k in eval_dict_last_epoch_trunc:
  del eval_dict_last_epoch_trunc[k]["val_metics_df"]
eval_dict_last_epoch_trunc


In [ ]:
with open("results/eval_last_epoch.txt", "w") as outfile:
    outfile.write(str(eval_dict_last_epoch))

with open("results/eval_best_epoch.txt", "w") as outfile:
    outfile.write(str(eval_dict_best_epoch))


In [ ]:
# def get_losses(filename):

#   # open the text file
#   with open(filename, 'r') as f:
#       contents = f.read()

#   # extract the train losses and val losses using regular expressions
#   train_losses_str = re.search(
#       r"losses:{'train': (.+?), 'val'", contents).group(1)
#   val_losses_str = re.search(r"'val': \[(.+?)\}", contents).group(1)

#   train_losses_str = train_losses_str.replace('[', '').replace(']', '')
#   val_losses_str = val_losses_str.replace('[', '').replace(']', '')

#   # convert the strings to lists of floating-point numbers
#   train_losses = [float(x.strip()) for x in train_losses_str.split(', ')]
#   val_losses = [float(x.strip()) for x in val_losses_str.split(', ')]

#   print(f"{len(train_losses)} train and {len(val_losses)} val losses found in {filename}")

#   # print the lists
#   return train_losses, val_losses


# def get_param_count(filename):

#   # open the text file
#   with open(filename, 'r') as f:
#       contents = f.read()

#   # extract the param count using regular expressions
#   param_str = re.search(
#       r"(?<=parameter_count:).*", contents).group(0)
#   param_count = int(param_str)
#   print(f"Parameter count = {param_count} in {filename}")

#   return param_count


# def get_dice_scores(filename):

#   # open the text file
#   with open(filename, 'r') as f:
#       lines = f.readlines()

#   for line in lines:
#     if "dice_scores" in line:

#       # extract the dice scores and iou scores using regular expressions
#       train_dice_str = re.search(
#           r"'train': (.+?), 'val'", line).group(1)
#       val_dice_str = re.search(r"'val': \[(.+?)\}", line).group(1)

#       train_dice_str = train_dice_str.replace('[', '').replace(']', '')
#       val_dice_str = val_dice_str.replace('[', '').replace(']', '')

#       # convert the strings to lists of floating-point numbers
#       train_dice = [float(x.strip()) for x in train_dice_str.split(', ')]
#       val_dice = [float(x.strip()) for x in val_dice_str.split(', ')]
#       break
#   print(f"{len(train_dice)} train and {len(val_dice)} val dice score found in {filename}")
#   return train_dice, val_dice


# def get_jaccard_scores(filename):

#   # open the text file
#   with open(filename, 'r') as f:
#       lines = f.readlines()

#   for line in lines:
#     if "jaccard_scores" in line:

#       # extract the dice scores and iou scores using regular expressions
#       train_jaccard_str = re.search(
#           r"'train': (.+?), 'val'", line).group(1)
#       val_jaccard_str = re.search(r"'val': \[(.+?)\}", line).group(1)

#       train_jaccard_str = train_jaccard_str.replace('[', '').replace(']', '')
#       val_jaccard_str = val_jaccard_str.replace('[', '').replace(']', '')

#       # convert the strings to lists of floating-point numbers
#       train_jaccard = [float(x.strip()) for x in train_jaccard_str.split(', ')]
#       val_jaccard = [float(x.strip()) for x in val_jaccard_str.split(', ')]
#       break
#   print(f"{len(train_jaccard)} train and {len(val_jaccard)} val jaccard score found in {filename}")
#   return train_jaccard, val_jaccard


# def get_train_run_time(filename):

#   # open the text file
#   with open(filename, 'r') as f:
#       contents = f.read()

#   # extract the param count using regular expressions
#   time_str = re.search(
#       r"(?<=last_completed_run_time:).*(?=\n)", contents).group(0)
#   time_obj = datetime.strptime(time_str, "%H:%M:%S.%f").time()
#   print(f"Trainer runtime = {time_obj} in {filename}")
#   return time_obj

from utils.viz_eval_utils import get_losses, get_param_count, get_dice_scores, get_jaccard_scores, get_train_run_time


FILENAME = "Logs/3DOnet_DoubleConv_Kernel1/trainer_properties.txt"

get_param_count(FILENAME)
get_train_run_time(FILENAME)
get_losses(FILENAME)
get_dice_scores(FILENAME)
get_jaccard_scores(FILENAME)
print("Test done")


In [ ]:
train_results_dict = {}

for dirname in os.listdir("Logs"):
  model_name = dirname
  train_results_dict[model_name] = {}

  filename = f"Logs/{dirname}/trainer_properties.txt"
  parameter_count = get_param_count(filename)
  trainer_runtime = get_train_run_time(filename)
  train_losses, val_losses = get_losses(filename)
  train_dices, val_dices = get_dice_scores(filename)
  train_jaccards, val_jacards = get_jaccard_scores(filename)
  train_results_dict[model_name]["parameter_count"] = parameter_count
  train_results_dict[model_name]["trainer_runtime"] = trainer_runtime
  train_results_dict[model_name]["train_losses"] = train_losses
  train_results_dict[model_name]["val_losses"] = val_losses
  train_results_dict[model_name]["train_dices"] = train_dices
  train_results_dict[model_name]["val_dices"] = val_dices
  train_results_dict[model_name]["train_jaccards"] = train_jaccards
  train_results_dict[model_name]["val_jacards"] = val_jacards


In [ ]:
# obtain color palette with shades corresponding to parameter counts

Unet_colors = ["#D8E8FA", "#CCE0F5", "#B2D2F7",
               "#91C2FA", "#69ADF9", "#3E97FA", "#117FFB"]  # blue
# Onet_colors = ["#F9E0ED", "#F9C7E1", "#F9A5D1", "#FB7CBE", "#FC57AC", "#FA2693", "#DE0977"] # pink
Onet_colors = ["#F4E6F7", "#F0CCF9", "#EBABF9",
               "#E483FA", "#DC4AFC", "#CF0CFA", "#9703B8"]  # purple
Swin_colors = ["#D3F3C4", "#A3F47D", "#44C10A"]  # green
spherenet_colors = ["#FC9A03"]  # orange
Unet_dict = {}
Onet_dict = {}
Swin_dict = {}
spherenet_dict = {}
for model_name in train_results_dict:
    if "Unet" in model_name:
        Unet_dict[model_name] = train_results_dict[model_name]["parameter_count"]
    elif "Onet" in model_name:
        Onet_dict[model_name] = train_results_dict[model_name]["parameter_count"]
    elif "Swin" in model_name:
        Swin_dict[model_name] = train_results_dict[model_name]["parameter_count"]
    else:
        spherenet_dict[model_name] = train_results_dict[model_name]["parameter_count"]

sorted_Unet_models = sorted(Unet_dict.items(), key=lambda x: x[1])
sorted_Onet_models = sorted(Onet_dict.items(), key=lambda x: x[1])
sorted_Swin_models = sorted(Swin_dict.items(), key=lambda x: x[1])
sorted_spherenet_models = sorted(spherenet_dict.items(), key=lambda x: x[1])

cols = {}
for i in range(len(sorted_Unet_models)):
    cols[sorted_Unet_models[i][0]] = Unet_colors[i]
for i in range(len(sorted_Onet_models)):
    cols[sorted_Onet_models[i][0]] = Onet_colors[i]
for i in range(len(sorted_Swin_models)):
    cols[sorted_Swin_models[i][0]] = Swin_colors[i]
for i in range(len(sorted_spherenet_models)):
    cols[sorted_spherenet_models[i][0]] = spherenet_colors[i]


In [ ]:
# def plot_param_count(results_dict, paletteCols):
#   x = []
#   y = []
#   for model_name in results_dict:
#     x.append(model_name)
#     y.append(results_dict[model_name]["parameter_count"])

#   # sort based on ascending order of parameter count
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)

#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Parameter Count")

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.set_xlabel("Models")
#   ax.set_ylabel("Parameter Count")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   plt.show()


# def plot_train_losses(results_dict, paletteCols):

#   epoch_num = len(results_dict["3DOnet_DoubleConv_Kernel1"]["train_losses"])
#   epoch_num_xlist = list(range(1, epoch_num + 1))
#   train_loss_data = {'Epochs': epoch_num_xlist}
#   train_loss_df = pd.DataFrame(train_loss_data)

#   for model_name in results_dict:
#     train_loss_df[model_name] = results_dict[model_name]["train_losses"]

#   models_ls = []
#   for model_name in results_dict:
#     models_ls.append(model_name)

#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Train Losses")
#   sns.lineplot(data=train_loss_df[models_ls], linewidth=3, palette=paletteCols)
#   ax.set_xlabel("Epochs")
#   ax.set_ylabel("Train Losses")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   plt.show()


# def plot_val_losses(results_dict, paletteCols):

#   epoch_num = len(results_dict["3DOnet_DoubleConv_Kernel1"]["val_losses"])
#   epoch_num_xlist = list(range(1, epoch_num + 1))
#   val_loss_data = {'Epochs': epoch_num_xlist}
#   val_loss_df = pd.DataFrame(val_loss_data)

#   for model_name in results_dict:
#     val_loss_df[model_name] = results_dict[model_name]["val_losses"]

#   models_ls = []
#   for model_name in results_dict:
#     models_ls.append(model_name)

#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Validation Losses")
#   sns.lineplot(data=val_loss_df[models_ls], linewidth=3, palette=paletteCols)
#   ax.set_xlabel("Epochs")
#   ax.set_ylabel("Validation Losses")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   plt.show()
  

# def plot_train_dices(results_dict, paletteCols):

#   epoch_num = len(results_dict["3DOnet_DoubleConv_Kernel1"]["train_dices"])
#   epoch_num_xlist = list(range(1, epoch_num + 1))
#   train_dices_data = {'Epochs': epoch_num_xlist}
#   train_dices_df = pd.DataFrame(train_dices_data)

#   for model_name in results_dict:
#     train_dices_df[model_name] = results_dict[model_name]["train_dices"]

#   models_ls = []
#   for model_name in results_dict:
#     models_ls.append(model_name)

#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Train Dice Scores")
#   sns.lineplot(data=train_dices_df[models_ls], linewidth=3, palette=paletteCols)
#   ax.set_xlabel("Epochs")
#   ax.set_ylabel("Train Dice Scores")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   plt.show()


# def plot_val_dices(results_dict, paletteCols):

#   epoch_num = len(results_dict["3DOnet_DoubleConv_Kernel1"]["val_dices"])
#   epoch_num_xlist = list(range(1, epoch_num + 1))
#   val_dices_data = {'Epochs': epoch_num_xlist}
#   val_dices_df = pd.DataFrame(val_dices_data)

#   for model_name in results_dict:
#     val_dices_df[model_name] = results_dict[model_name]["val_dices"]

#   models_ls = []
#   for model_name in results_dict:
#     models_ls.append(model_name)

#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Validation Dice Scores")
#   sns.lineplot(data=val_dices_df[models_ls], linewidth=3, palette=paletteCols)
#   ax.set_xlabel("Epochs")
#   ax.set_ylabel("Validation Dice Scores")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   plt.show()
  

# def plot_train_jaccards(results_dict, paletteCols):

#   epoch_num = len(results_dict["3DOnet_DoubleConv_Kernel1"]["train_jaccards"])
#   epoch_num_xlist = list(range(1, epoch_num + 1))
#   train_jaccards_data = {'Epochs': epoch_num_xlist}
#   train_jaccards_df = pd.DataFrame(train_jaccards_data)

#   for model_name in results_dict:
#     train_jaccards_df[model_name] = results_dict[model_name]["train_jaccards"]

#   models_ls = []
#   for model_name in results_dict:
#     models_ls.append(model_name)

#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Train Jaccard Scores")
#   sns.lineplot(data=train_jaccards_df[models_ls], linewidth=3, palette=paletteCols)
#   ax.set_xlabel("Epochs")
#   ax.set_ylabel("Train Jaccard Scores")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   plt.show()


# def plot_val_jacards(results_dict, paletteCols):

#   epoch_num = len(results_dict["3DOnet_DoubleConv_Kernel1"]["val_jacards"])
#   epoch_num_xlist = list(range(1, epoch_num + 1))
#   val_jacards_data = {'Epochs': epoch_num_xlist}
#   val_jacards_df = pd.DataFrame(val_jacards_data)

#   for model_name in results_dict:
#     val_jacards_df[model_name] = results_dict[model_name]["val_jacards"]

#   models_ls = []
#   for model_name in results_dict:
#     models_ls.append(model_name)

#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Validation Jaccard Scores")
#   sns.lineplot(data=val_jacards_df[models_ls], linewidth=3, palette=paletteCols)
#   ax.set_xlabel("Epochs")
#   ax.set_ylabel("Validation Jaccard Scores")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   plt.show()

# def plot_trainer_runtime(results_dict, paletteCols):
#   x = []
#   y = []
#   for model_name in results_dict:
#     x.append(model_name)
#     y.append(results_dict[model_name]["trainer_runtime"])

#   # sort based on ascending order of parameter count
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)
#   y = [round(i.hour+(i.minute/60), 2) for i in y]
#   # today = datetime.datetime.now()
#   # y = [datetime.datetime.combine(today, t) for t in y]
#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("Model Trainer Runtime (hours)")

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.bar_label(ax.containers[0])
#   # plt.bar(x, y)
#   ax.set_xlabel("Models")
#   ax.set_ylabel("Trainer runtime in hours")
#   ax.legend()
#   plt.xticks(rotation=45, ha='right')
#   fig.savefig("results/trainer_runtime_all.png", format="png",
#               pad_inches=0.2, transparent=False, bbox_inches='tight')
#   plt.show()


from utils.viz_eval_utils import plot_param_count, plot_train_losses, plot_val_losses, plot_train_dices, plot_val_dices, plot_train_jaccards, plot_val_jacards, plot_trainer_runtime

plot_train_losses(results_dict=train_results_dict, paletteCols=cols)
plot_val_losses(results_dict=train_results_dict, paletteCols=cols)
plot_train_dices(results_dict=train_results_dict, paletteCols=cols)
plot_val_dices(results_dict=train_results_dict, paletteCols=cols)
plot_train_jaccards(results_dict=train_results_dict, paletteCols=cols)
plot_val_jacards(results_dict=train_results_dict, paletteCols=cols)
plot_param_count(results_dict=train_results_dict, paletteCols=cols)
plot_trainer_runtime(results_dict=train_results_dict, paletteCols=cols)


In [ ]:
from utils.viz_eval_utils import plot_WT_dice, plot_WT_jaccard, plot_TC_dice, plot_TC_jaccard, plot_ET_dice, plot_ET_jaccard, plot_inference_time

# def plot_WT_dice(results_dict, paletteCols):
#   mean_scores_dict = {
#       k: results_dict[k]["WT dice"] for k in results_dict}
#   x = []
#   y = []
#   for model_name in mean_scores_dict:
#     x.append(model_name)
#     y.append(round(mean_scores_dict[model_name],3))
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)
#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("WT Dice Scores")
#   # plt.bar(x, y)

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.bar_label(ax.containers[0])
#   ax.bar_label(ax.containers[0])

#   ax.set_xlabel("Models")
#   ax.set_ylabel("WT Dice")
#   ax.set_ylim(0.5, 0.9)
#   plt.xticks(rotation=45, ha='right')
#   fig.savefig("results/WT_dice_all.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
#   plt.show()
  

# def plot_WT_jaccard(results_dict, paletteCols):
#   mean_scores_dict = {
#       k: results_dict[k]["WT jaccard"] for k in results_dict}
#   x = []
#   y = []
#   for model_name in mean_scores_dict:
#     x.append(model_name)
#     y.append(round(mean_scores_dict[model_name],3))
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)
#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("WT Jaccard Scores")
#   # plt.bar(x, y)

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.bar_label(ax.containers[0])

#   ax.set_xlabel("Models")
#   ax.set_ylabel("WT Jaccard")
#   ax.set_ylim(0.5, 0.9)
#   plt.xticks(rotation=45, ha='right')
#   fig.savefig("results/WT_jaccard_all.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
#   plt.show()
  

# def plot_TC_dice(results_dict, paletteCols):
#   mean_scores_dict = {
#       k: results_dict[k]["TC dice"] for k in results_dict}
#   x = []
#   y = []
#   for model_name in mean_scores_dict:
#     x.append(model_name)
#     y.append(round(mean_scores_dict[model_name],3))
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)
#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("TC Dice Scores")
#   # plt.bar(x, y)

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.bar_label(ax.containers[0])

#   ax.set_xlabel("Models")
#   ax.set_ylabel("TC Dice")
#   ax.set_ylim(0.5, 0.9)
#   plt.xticks(rotation=45, ha='right')
#   fig.savefig("results/TC_dice_all.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
#   plt.show()
  

# def plot_TC_jaccard(results_dict, paletteCols):
#   mean_scores_dict = {
#       k: results_dict[k]["TC jaccard"] for k in results_dict}
#   x = []
#   y = []
#   for model_name in mean_scores_dict:
#     x.append(model_name)
#     y.append(round(mean_scores_dict[model_name],3))
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)
#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("TC Jaccard Scores")
#   # plt.bar(x, y)

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.bar_label(ax.containers[0])

#   ax.set_xlabel("Models")
#   ax.set_ylabel("TC Jaccard")
#   ax.set_ylim(0.5, 0.9)
#   plt.xticks(rotation=45, ha='right')
#   fig.savefig("results/TC_jaccard_all.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
#   plt.show()


# def plot_ET_dice(results_dict, paletteCols):
#   mean_scores_dict = {
#       k: results_dict[k]["ET dice"] for k in results_dict}
#   x = []
#   y = []
#   for model_name in mean_scores_dict:
#     x.append(model_name)
#     y.append(round(mean_scores_dict[model_name],3))
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)
#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("ET Dice Scores")
#   # plt.bar(x, y)

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.bar_label(ax.containers[0])

#   ax.set_xlabel("Models")
#   ax.set_ylabel("ET Dice")
#   ax.set_ylim(0.5, 0.9)
#   plt.xticks(rotation=45, ha='right')
#   fig.savefig("results/ET_dice_all.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
#   plt.show()
  

# def plot_ET_jaccard(results_dict, paletteCols):
#   mean_scores_dict = {
#       k: results_dict[k]["ET jaccard"] for k in results_dict}
#   x = []
#   y = []
#   for model_name in mean_scores_dict:
#     x.append(model_name)
#     y.append(round(mean_scores_dict[model_name],3))
#   x = [val for _, val in sorted(zip(y, x))]
#   y = sorted(y)
#   fig, ax = plt.subplots(figsize=(12, 8))
#   ax.set_title("ET Jaccard Scores")
#   # plt.bar(x, y)

#   sns.barplot(x=x, y=y, palette=paletteCols)
#   ax.bar_label(ax.containers[0])

#   ax.set_xlabel("Models")
#   ax.set_ylabel("ET Jaccard")
#   ax.set_ylim(0.5, 0.9)
#   plt.xticks(rotation=45, ha='right')
#   fig.savefig("results/ET_jaccard_all.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
#   plt.show()


# def plot_inference_time(results_dict, paletteCols):
#     inferenceDict = {
#         k: results_dict[k]["Inference time"] for k in results_dict}
#     # print(inferenceDict)
#     x = []
#     y = []
#     for model_name in inferenceDict:
#         x.append(model_name)
#         y.append(inferenceDict[model_name])

#     # sort based on ascending order of parameter count
#     x = [val for _, val in sorted(zip(y, x))]
#     y = sorted(y)
#     y = [round(i.seconds + i.microseconds*1e-6, 3) for i in y]

#     fig, ax = plt.subplots(figsize=(12, 8))
#     ax.set_title("Inference time for 53 samples (seconds)")

#     sns.barplot(x=x, y=y, palette=paletteCols)
#     ax.bar_label(ax.containers[0])
#     ax.set_xlabel("Models")
#     ax.set_ylabel("Inference time in seconds")
#     ax.legend()
#     ax.set_ylim(15, 40)
#     plt.xticks(rotation=45, ha='right')
#     fig.savefig("results/inference_time_all.png", format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
#     plt.show()





plot_WT_dice(eval_dict_last_epoch, cols)
plot_WT_jaccard(eval_dict_last_epoch, cols)
plot_TC_dice(eval_dict_last_epoch, cols)
plot_TC_jaccard(eval_dict_last_epoch, cols)
plot_ET_dice(eval_dict_last_epoch, cols)
plot_ET_jaccard(eval_dict_last_epoch, cols)
plot_inference_time(eval_dict_last_epoch, cols)
# plot_trainer_runtime(train_results_dict, cols)


In [ ]:
eval_dict_last_epoch["3DOnet_DoubleConv_Kernel1"]['val_metics_df']
for k in eval_dict_last_epoch:
  eval_dict_last_epoch[k]['val_metics_df'] = eval_dict_last_epoch[k]['val_metics_df'].to_dict()


In [ ]:
x = []
y = []
for k in train_results_dict:
  x.append(k)
  y.append(train_results_dict[k]["parameter_count"])
  # parameter_count_df.loc[k] = results_dict[k]["parameter_count"]
data_tuple = list(zip(x, y))
parameter_count_df = pd.DataFrame(
    data_tuple, columns=['Model', 'Params (1e6)']).set_index("Model")
parameter_count_df = parameter_count_df.applymap(lambda x: round(x*1e-6,2))
parameter_count_df


In [ ]:

x = []
y = []
for k in train_results_dict:
  x.append(k)
  y.append(train_results_dict[k]["trainer_runtime"])
  # parameter_count_df.loc[k] = results_dict[k]["parameter_count"]
y = [round(i.hour+(i.minute/60),3) for i in y]
data_tuple = list(zip(x, y))
trainer_runtime_df = pd.DataFrame(
    data_tuple, columns=['Model', 'Train time (h)']).set_index("Model")
# trainer_runtime_df = trainer_runtime_df.applymap(lambda x: round(x*1e-6, 2))
trainer_runtime_df


In [ ]:
results_df = pd.DataFrame()
# for k in eval_dict_last_epoch:
results_df = pd.DataFrame.from_dict(eval_dict_last_epoch)
results_df = results_df.transpose().sort_index()
results_rounded = results_df
results_rounded["Inference time"] = results_rounded["Inference time"].apply(
    lambda x: float(np.format_float_positional(x.seconds + x.microseconds*1e-6, precision=4, unique=False, fractional=False, trim='k')))
# results_df.applymap('{:,.2f}'.format)
# results_rounded
results_rounded = results_rounded.applymap(lambda x: float(
    np.format_float_positional(x, precision=4, unique=False, fractional=False, trim='k')))

results_rounded = results_rounded.join(trainer_runtime_df)
results_rounded = results_rounded.join(parameter_count_df)

results_rounded.rename(
    columns={'Inference time': 'Infer time (s)'}, inplace=True)
results_rounded.to_csv("results/full_results_statistics.csv")
results_rounded


In [ ]:
print(results_rounded.to_latex())


In [ ]:
def save_eval_csv_per_model_in_log():
    for k in eval_dict_last_epoch:
        print(k)
        sample_results = pd.DataFrame.from_dict(
            eval_dict_last_epoch[k]["val_metics_df"])
        sample_results.to_csv(f"Logs/{k}/eval_results.csv")
        


save_eval_csv_per_model_in_log()

In [ ]:
def compare_predictions_in_results(target="BraTS20_Training_004", treshold=0.5):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'

  name, imgs, targets = None, None, None
  _, _, test_dataloader = get_dataloaders(
      dataset=BratsDataset, path_to_csv="./fold_data.csv", val_fold=0, test_fold=1, batch_size=1, do_resizing=True)

  for i, data in enumerate(test_dataloader):
    if data['Id'][0] == target:
      print("Target found: ", target)
      name, imgs, targets = data['Id'][0], data['image'], data['mask']
      
  fig, ax_array = plt.subplots(len(os.listdir("Logs")), 7, figsize=(20, 50))
  
  rows = []
  cols = ['img', 'WT_GT', 'WT_Pred', 'TC_GT', 'TC_Pred', 'ET_GT', 'ET_Pred']
  for i, dirname in enumerate(os.listdir("Logs")):
    
    model_name = dirname
    rows.append(model_name)
    model = modelDict[model_name]
    checkpoint_path = None

    for filename in os.listdir(f"Logs/{dirname}"):
      if "your_last_epoch_model" in filename:
        checkpoint_path = f"Logs/{dirname}/{filename}"
        break

    try:
      model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'))
      model.eval()
      print(f"{model_name} loaded with chkpt: {checkpoint_path}. parameters: {count_parameters(model)}")
    except Exception as e:
      print(
          f"Error loading {model_name} with chkpt: {checkpoint_path}. parameters: {count_parameters(model)}")
      print(e)

    with torch.no_grad():
      print("Predicting for: ", model_name)
      imgs, targets = imgs.to(device), targets.to(device)
      logits = model(imgs)
      probs = torch.sigmoid(logits)

      predictions = (probs >= treshold).float()
      predictions = predictions.cpu()
      targets = targets.cpu()

      img, gt, prediction = imgs.cpu(), targets, predictions

      img = img.squeeze()[0].cpu().detach().numpy()
      gt0 = gt[0][0].squeeze().cpu().detach().numpy()
      gt1 = gt[0][1].squeeze().cpu().detach().numpy()
      gt2 = gt[0][2].squeeze().cpu().detach().numpy()
      pred0 = prediction[0][0].squeeze().cpu().detach().numpy()
      pred1 = prediction[0][1].squeeze().cpu().detach().numpy()
      pred2 = prediction[0][2].squeeze().cpu().detach().numpy()

      
      slice_w = 75
      
      ax_array[i][0].imshow(img[slice_w, :, :], cmap="gray")
      ax_array[i][0].tick_params(axis='both', length=0, width=0)
      ax_array[i][0].set_xlabel('')
      ax_array[i][0].set_ylabel('')
      ax_array[i][1].imshow(gt0[slice_w, :, :], cmap="viridis")
      ax_array[i][1].tick_params(axis='both', length=0, width=0)
      ax_array[i][1].set_xlabel('')
      ax_array[i][1].set_ylabel('')
      ax_array[i][2].imshow(pred0[slice_w, :, :], cmap="viridis")
      ax_array[i][2].tick_params(axis='both', length=0, width=0)
      ax_array[i][2].set_xlabel('')
      ax_array[i][2].set_ylabel('')
      ax_array[i][3].imshow(gt1[slice_w, :, :], cmap="cividis")
      ax_array[i][3].tick_params(axis='both', length=0, width=0)
      ax_array[i][3].set_xlabel('')
      ax_array[i][3].set_ylabel('')
      ax_array[i][4].imshow(pred1[slice_w, :, :], cmap="cividis")
      ax_array[i][4].tick_params(axis='both', length=0, width=0)
      ax_array[i][4].set_xlabel('')
      ax_array[i][4].set_ylabel('')
      ax_array[i][5].imshow(gt2[slice_w, :, :], cmap="plasma")
      ax_array[i][5].tick_params(axis='both', length=0, width=0)
      ax_array[i][5].set_xlabel('')
      ax_array[i][5].set_ylabel('')
      ax_array[i][6].imshow(pred2[slice_w, :, :], cmap="plasma")
      ax_array[i][6].tick_params(axis='both', length=0, width=0)
      ax_array[i][6].set_xlabel('')
      ax_array[i][6].set_ylabel('')

    del model
  for ax, col in zip(ax_array[0], cols):
    ax.set_title(col)
  for ax, row in zip(ax_array[:, 0], rows):
    ax.set_ylabel(row, rotation=90, size='small')
  fig.savefig(f"results/prediction_comparison.png",
              format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
  print(
      f"Saving to results/prediction_comparison.png")
  plt.show()
  # fig.suptitle(f'Model: {model_name} | Sample: {name} | Slice: {slice_w}', y=0.7, fontsize=16)


compare_predictions_in_results()


In [ ]:
def save_predictions_to_logs(target="BraTS20_Training_004", treshold=0.5):
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  
  name, imgs, targets = None, None, None
  _, _, test_dataloader = get_dataloaders(
      dataset=BratsDataset, path_to_csv="./fold_data.csv", val_fold=0, test_fold=1, batch_size=1, do_resizing=True)
  
  for i, data in enumerate(test_dataloader):
    if data['Id'][0] == target:
      print("Target found: ", target)
      name, imgs, targets = data['Id'][0], data['image'], data['mask']
  
  for dirname in os.listdir("Logs"):
    model_name = dirname
    model = modelDict[model_name]
    savepath = f"Logs/{dirname}/prediction_samples"
    if not os.path.isdir(savepath):
      os.mkdir(savepath)
    results = {"Id": [], "image": [], "GT": [], "Prediction": []}

    checkpoint_path = None

    for filename in os.listdir(f"Logs/{dirname}"):
      if "your_last_epoch_model" in filename:
        checkpoint_path = f"Logs/{dirname}/{filename}"
        break

    try:
      model.load_state_dict(torch.load(checkpoint_path, map_location='cpu'))
      model.eval()
      print(f"{model_name} loaded with chkpt: {checkpoint_path}. parameters: {count_parameters(model)}")
    except Exception as e:
      print(f"Error loading {model_name} with chkpt: {checkpoint_path}. parameters: {count_parameters(model)}")
      print(e)

    with torch.no_grad():
      print("Predicting for: ", model_name)
      imgs, targets = imgs.to(device), targets.to(device)
      logits = model(imgs)
      probs = torch.sigmoid(logits)

      predictions = (probs >= treshold).float()
      predictions = predictions.cpu()
      targets = targets.cpu()
      
      img, gt, prediction = imgs.cpu(), targets, predictions
      
      img = img.squeeze()[0].cpu().detach().numpy()
      gt0 = gt[0][0].squeeze().cpu().detach().numpy()
      gt1 = gt[0][1].squeeze().cpu().detach().numpy()
      gt2 = gt[0][2].squeeze().cpu().detach().numpy()
      pred0 = prediction[0][0].squeeze().cpu().detach().numpy()
      pred1 = prediction[0][1].squeeze().cpu().detach().numpy()
      pred2 = prediction[0][2].squeeze().cpu().detach().numpy()
      
      for i in range(50, 90+1, 5):

        fig, (ax1, ax2, ax3, ax4, ax5, ax6, ax7) = plt.subplots(
            1, 7, figsize=(30, 10))
        slice_w = i
        
        ax1.imshow(img[slice_w, :, :], cmap="gray")
        ax1.set_title(f'img')
        ax2.imshow(gt0[slice_w, :, :], cmap="viridis")
        ax2.set_title(f'WT_GT')
        ax3.imshow(pred0[slice_w, :, :], cmap="viridis")
        ax3.set_title(f'WT_Pred')
        ax4.imshow(gt1[slice_w, :, :], cmap="cividis")
        ax4.set_title(f'TC_GT')
        ax5.imshow(pred1[slice_w, :, :], cmap="cividis")
        ax5.set_title(f'TC_Pred')
        ax6.imshow(gt2[slice_w, :, :], cmap="plasma")
        ax6.set_title(f'ET_GT')
        ax7.imshow(pred2[slice_w, :, :], cmap="plasma")
        ax7.set_title(f'ET_Pred')
        # fig.tight_layout(rect=[0, 0.03, 1, 0.95])
        fig.suptitle(
            f'Model: {model_name} | Sample: {name} | Slice: {i}', y=0.7, fontsize=16)
        # plt.show()
        fig.savefig(f"{savepath}/prediction_{name}_slice_{i}.png",
                    format="png",  pad_inches=0.2, transparent=False, bbox_inches='tight')
        print(
            f"Saving to {savepath}/prediction_{name}_slice_{i}.png")
    del model
  

save_predictions_to_logs()